In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.animation as animation
import datetime
%load_ext tensorboard

In [2]:
class CRNN(tf.Module):
    def __init__(self, p_init, name="CRNN"):
        super(CRNN, self).__init__(name=name)

        #设置工作目录
        self.expr_name = "4s8r-01"
        self.sys_path = "W:\CRNN-TF2.0\CRNN-TF2.0"
        self.fig_path = str(self.sys_path + "/results/" + self.expr_name + "/figs/")
        self.ckpt_path = str(self.sys_path + "/results/" + self.expr_name + "/checkpoint/")
        self.script_path = str(self.sys_path + "/scripts/")
        self.exp_path = str(self.sys_path + "/exp_data/")

        #设置初始参数
        self.lb = 1e-8
        self.n_epoch = 2500
        self.n_plot = 10
        self.grad_max = 1e2
        self.maxiters = 50000

        self.lr_max = 5e-3
        self.lr_min = 1e-5
        self.lr_decay = 0.2
        self.lr_decay_step = 500
        self.w_decay = 1e-8

        self.llb = self.lb
        self.p_cutoff = -1.0
        self.R = -1.0 / 8.314e-3

        self.l_exp = range(1,15)
        self.n_exp = 14
        self.l_train = []
        self.l_val = []
        for i in np.arange(1,self.n_exp+1):
            j = self.l_exp[i-1]
            if not (j in [2, 6, 9, 12]):
                self.l_train.append(i)
            else:
                self.l_val.append(i)

        #导入数据为内置tensor.constant
        self.exp_data, self.T0, self.beta, self.ocen = self.load_data()

        #设置用到的实验数据
        self.current_iexp = 1
        self.select_exp(self.current_iexp)

        #########################################
        self.nr=8
        self.ns=4
        self.npara = self.nr * (self.ns + 4) + 1
        
        #p = np.random.randn(npara) * 1.e-2
        #p[:nr] += 0.8 # w_b
        # nr~nr*(ns+1) vij
        #p[nr*(ns+1):nr*(ns+2)] += 0.8 # w_out
        #p[nr*(ns+2):nr*(ns+4)] += 0.1 # w_b | w_Ea
        #p[-1] = 0.1 # slope
        self.p = tf.Variable(p_init, name="p", dtype=tf.float64)
        self.update_p(p)

        u0 = np.zeros(self.ns)
        u0[0] = 1
        self.u0 = tf.constant(u0, shape=(self.ns,),dtype=tf.float64)
        

    # 导入文件
    def load_data(self):
        T0 = np.zeros(self.n_exp)
        beta = np.zeros(self.n_exp)
        ocen = np.zeros(self.n_exp)
        exp_data = []
        for i_exp, value in enumerate(self.l_exp):
            filename = str(self.exp_path + "expdata_no"+str(value)+".txt")
            exp_npdata = np.loadtxt(filename)
            exp_npdata[:, 2] = exp_npdata[:, 2] / max(exp_npdata[:,2])
            if value == 4:
                exp_npdata = exp_npdata[:60, :]
            elif value == 5:
                exp_npdata = exp_npdata[:58, :]
            elif value == 6:
                exp_npdata = exp_npdata[:60, :]
            elif value == 7:
                exp_npdata = exp_npdata[:71, :]
            T0[i_exp-1] = exp_npdata[0, 1]
            exp_tfdata = tf.convert_to_tensor(exp_npdata, dtype=tf.float64)
            exp_data.append(exp_tfdata)
        T0 = tf.convert_to_tensor(T0, dtype=tf.float64)
        beta = tf.convert_to_tensor(np.loadtxt(self.exp_path + "beta.txt"), dtype=tf.float64)
        ocen = tf.convert_to_tensor(np.log(np.loadtxt(self.exp_path + "ocen.txt")+self.llb), dtype=tf.float64)
        return exp_data, T0, beta, ocen

    # 更新p
    def update_p(self, p):
        self.p = p
        self.w_in, self.w_b, self.w_out = self.p2vec(p)
    # 选择要用到的实验数据，将其整理为纯净的tf Tensor
    def select_exp(self, i_exp):
        self.current_T0 = tf.reshape(self.T0[i_exp-1:i_exp], shape=(1,1))
        self.current_exp_data = self.exp_data[i_exp-1]
        self.current_beta = tf.reshape(self.beta[i_exp-1:i_exp], shape=(1,1))
        self.current_ocen = tf.reshape(self.ocen[i_exp-1:i_exp], shape=(1,1))
        self.current_ts = self.current_exp_data[:,0]
        self.current_tinit = tf.constant(self.current_ts[0], dtype=tf.float64)

    #拆分权重向量
    @tf.function
    def p2vec(self, p):
        slope = p[-1] * 1.e1
        
        w_b = p[:self.nr] * (slope * 10.0)
        w_b = tf.clip_by_value(w_b, clip_value_min=0., clip_value_max=50.)
        
        # julia 与 tf的reshape函数实现不同
        w_out = tf.reshape(p[self.nr:self.nr*(self.ns+1)],shape=(self.nr,self.ns))
        w_out = tf.transpose(w_out)

        temp1 = tf.clip_by_value(w_out[0:1, :], -3., 0.)
        temp2 = tf.clip_by_value(tf.abs(w_out[-1:, :]), 0., 3.)
        w_out = tf.concat([temp1, w_out[1:-1], temp2], axis=0)

        # 此段省略
        # julia原文：
        # if p_cutoff > 0.0
        #     w_out[findall(abs.(w_out) .< p_cutoff)] .= 0.0
        # end

        temp3 = -1*(tf.reduce_sum(w_out[:-2,:], 0) + w_out[-1:, :])
        temp3 = tf.reshape(temp3, shape=(1,self.nr))
        
        w_out = tf.concat([w_out[:-2,:], temp3, w_out[-1:,:]], axis=0)

        w_in_Ea = tf.abs(p[self.nr*(self.ns+1):self.nr*(self.ns+2)]) * slope * 100.
        w_in_Ea = tf.clip_by_value(w_in_Ea, 0., 300.)
        w_in_Ea = tf.reshape(w_in_Ea, shape=(1,8))

        w_in_b = tf.abs(p[self.nr*(self.ns+2):self.nr*(self.ns+3)])
        w_in_b = tf.reshape(w_in_b, shape=(1,8))

        w_in_ocen = tf.abs(p[self.nr*(self.ns+3):self.nr*(self.ns+4)])
        w_in_ocen = tf.clip_by_value(w_in_ocen, 0., 1.5)
        w_in_ocen = tf.reshape(w_in_ocen, shape=(1,8))

        # 此段省略
        # julia原文：
        # if p_cutoff > 0:
        #     w_in_ocean[abs(w_in_ocean)<p_cutoff] = 0.0
        
        w_in = tf.concat([
            tf.clip_by_value(w_out*(-1),0.,4.),
            w_in_Ea,
            w_in_b,
            w_in_ocen
        ], 0)

        return w_in, w_b, w_out

    # crnn计算方法
    @tf.function
    def crnn(self, t, u, p):
        self.update_p(p)
        logX = tf.dtypes.cast(tf.math.log(tf.reshape(tf.clip_by_value(u, self.lb, 10.), shape=(4,1))),tf.float64)
        T = self.getsampletemp(t)
        w_in_X = tf.matmul(tf.transpose(self.w_in), tf.concat([logX, self.R/T, tf.math.log(T), self.current_ocen],0))
        du_dt = tf.matmul(self.w_out, tf.exp(tf.reshape(w_in_X, shape=(self.nr,1))+tf.reshape(self.w_b, shape=(self.nr,1))))
        return du_dt

    # 工具函数
    @tf.function
    def getsampletemp(self, t):
        t = tf.dtypes.cast(t, tf.float64)
 
        T1 = self.current_T0 + t * (self.current_beta/60.0)
        HR = tf.constant(40.0/60.0, dtype=tf.float64)
        T2 = self.current_beta + 273.0
        
        temp1 = self.current_beta + 273.0 + HR * (t - 999.0*60.0)
        temp2 = tf.constant(370.0+273.0, dtype=tf.float64)
        T3 = tf.cond(tf.greater(temp1, temp2), lambda:temp2, lambda:temp1)
 
        temp3 = 370.0+273.0 + HR * (t-1059.0*60.0)
        temp4 = tf.constant(500.0+273.0, dtype=tf.float64)
        T4 = tf.cond(tf.greater(temp3, temp4), lambda:temp4, lambda:temp3)
        
        T_branch1 = tf.cond(tf.greater(t, 1059. * 60.), lambda: T4, lambda: T3)
        T_branch2 = tf.cond(tf.greater(t, 999. * 60.), lambda: T_branch1, lambda:T2)
        T = tf.cond(tf.greater(self.current_beta, 100), lambda: T_branch2, lambda: T1)

        return tf.reshape(T,shape=(1,1))

    @tf.function
    def pred_n_ode(self, p):
        solution = tf.function(lambda: tfp.math.ode.BDF().solve(
            self.crnn,
            self.current_tinit,#t_init
            self.u0,
            solution_times=self.current_ts,
            constants={'p': p}
        ), autograph=True)()
        return solution.states

    @tf.function
    def loss_n_ode(self, p):
        u_pred = self.pred_n_ode(p)
        
        masslist = tf.reduce_sum(tf.clip_by_value(u_pred[:-1,:], 0, 1e3),axis=0)
        gaslist = tf.clip_by_value(u_pred[-1:,:], 0, 1e3)

        loss = tf.keras.losses.MAE(masslist, self.current_exp_data[:tf.size(masslist),2])
        loss = tf.cond(tf.greater(self.current_ocen, 1000.0), lambda: loss, \
            lambda:loss + tf.keras.losses.MAE(gaslist, (1-self.current_exp_data[:tf.size(masslist),2])))
        return loss
        

In [1]:
#p = tf.Variable(np.loadtxt("p.txt"), dtype=tf.float64)
nr=8
ns=4
npara = nr * (ns + 4) + 1
p = np.random.randn(npara) * 1.e-2
p[:nr] += 0.8 # w_b
# nr~nr*(ns+1) vij
p[nr*(ns+1):nr*(ns+2)] += 0.8 # w_out
p[nr*(ns+2):nr*(ns+4)] += 0.1 # w_b | w_Ea
p[-1] = 0.1 # slope

p = tf.Variable(p, dtype=tf.float64)
modeltest = CRNN(p)
opt = tf.keras.optimizers.SGD(learning_rate=0.03)

NameError: name 'np' is not defined

In [5]:
grad_norm = tf.zeros(shape=(modeltest.n_exp,), dtype=tf.float64)
grad_norm = tf.Variable(grad_norm)
exp_list = np.array(modeltest.l_exp)
np.random.shuffle(exp_list)
for epoches in range(100):
    for i_exp in exp_list:
        if i_exp in modeltest.l_val:
            continue

        with tf.GradientTape() as tape:
            modeltest.select_exp(i_exp)
            tape.watch(p)
            modeltest.update_p(p)
            loss = modeltest.loss_n_ode(p)
        
        grad = tape.gradient(loss, p)
        grad_norm_val = tf.norm(grad, ord=2)
        grad = tf.cond(tf.greater(grad_norm_val, modeltest.grad_max), lambda: (grad / grad_norm_val) * modeltest.grad_max, lambda: grad)
        opt.apply_gradients(zip([grad], [p]))
    print(epoches, loss)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
0 tf.Tensor(
[23.52602274 23.52594089 23.52580028 23.52568981 23.52556986 23.52545896
 23.52535718 23.52523863 23.52512243 23.52505017 23.52498742 23.52491737
 23.52479439 23.52472008 23.5246304  23.52450188 23.524439   23.5243243
 23.52419764 23.5240485  23.52390281 23.52379693 23.52361133 23.52347325
 23.52329099 23.52308016 23.52287183 23.52268013 23.52242034 23.52210548
 23.52179544 23.52147178 23.52116682 23.52088682 23.52054051 23.52018938
 23.51983813 23.51947325 23.51908104 23.51871613 23.5183697  23.5179856
 23.51761559 23.51723656 23.51685744 23.51648121 23.51610489 23.5157285
 23.51535203 23.51497549 23.5145612  23.51418451 23.51380776 23.51343094
 23.5130464  23.51340449 23.51381919 23.51419625 23.51457337 23.51495054
 23.51532775 2

In [16]:
with tf.GradientTape() as tape:
    modeltest.select_exp(i_exp)
    tape.watch(p)
    modeltest.update_p(p)
    loss = modeltest.loss_n_ode(p)

Tensor("Sum:0", shape=(4,), dtype=float64)


In [6]:
loss

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([14.50651871])>